In [2]:
!pip install bs4


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the Solscan account balance changes page
url = "https://solscan.io/account/HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K#balanceChanges"

# Fetch the page content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Successfully fetched the page content.")
else:
    print("Failed to fetch the page content. Status code:", response.status_code)

Failed to fetch the page content. Status code: 403


In [4]:
import requests
import pandas as pd

# Solscan API endpoint for account data
api_url = "https://public-api.solscan.io/account/tokens"
params = {
    "account": "HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K"
}

# Add API key if you have one
headers = {
    "accept": "application/json",
    # "token": "your-api-key"  # Uncomment and add your API key if you have one
}

response = requests.get(api_url, params=params, headers=headers)

if response.status_code == 200:
    data = response.json()
    # Convert to DataFrame
    df = pd.DataFrame(data)
    print(df)
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

Failed to fetch data. Status code: 404


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# URL of the Solscan account balance changes page
url = "https://solscan.io/account/HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K#balanceChanges"

# Add headers to mimic a browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
}

try:
    # Add a small delay to be respectful to the server
    time.sleep(1)
    
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for bad status codes
    
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the table with balance changes
    # You'll need to inspect the page source to get the correct class or ID
    table = soup.find('table', {'class': 'table'})  # Adjust the class name as needed
    
    if table:
        # Extract table headers
        headers = []
        for th in table.find_all('th'):
            headers.append(th.text.strip())
        
        # Extract table rows
        rows = []
        for tr in table.find_all('tr')[1:]:  # Skip header row
            row = []
            for td in tr.find_all('td'):
                row.append(td.text.strip())
            if row:  # Only append non-empty rows
                rows.append(row)
        
        # Create DataFrame
        df = pd.DataFrame(rows, columns=headers)
        print(df)
        
        # Optionally save to CSV
        df.to_csv('balance_changes.csv', index=False)
        print("Data saved to balance_changes.csv")
        
    else:
        print("Could not find the table on the page")

except requests.exceptions.RequestException as e:
    print(f"Error fetching the page: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

Could not find the table on the page


In [8]:
import requests
import time
import json

# Base URL for Solscan with #balanceChanges
account_address = "HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K"
api_endpoint = f"https://api.solscan.io/account/exportTokenChanges"  # Changed to token changes endpoint

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'application/json',
    'Accept-Language': 'en-US,en;q=0.5',
    'Origin': 'https://solscan.io',
    'Referer': f'https://solscan.io/account/{account_address}#balanceChanges'
}

params = {
    'address': account_address,
    'type': 'tokenchange'  # Changed to token change type
}

try:
    # Create a session to maintain cookies
    with requests.Session() as session:
        session.headers.update(headers)
        # First load the main page to get any necessary cookies
        session.get(f"https://solscan.io/account/{account_address}#balanceChanges")
        time.sleep(1)  # Small delay to mimic human behavior
        
        # Then try the export
        response = session.get(api_endpoint, params=params)
        response.raise_for_status()
        
        if response.status_code == 200:
            with open('solscan_balance_changes.csv', 'wb') as f:
                f.write(response.content)
            print("Successfully downloaded CSV file as 'solscan_balance_changes.csv'")
        else:
            print(f"Failed to download CSV. Status code: {response.status_code}")
            
except requests.exceptions.RequestException as e:
    print(f"Error occurred: {e}")

Error occurred: 522 Server Error:  for url: https://api.solscan.io/account/exportTokenChanges?address=HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K&type=tokenchange


In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import os
import time
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import os
import time
from datetime import datetime

class SolscanScraper:
    def __init__(self, download_dir=None):
        self.download_dir = download_dir or os.path.join(os.getcwd(), "downloads")
        self.setup_download_directory()
        self.driver = None
        
    def setup_download_directory(self):
        """Create download directory if it doesn't exist"""
        if not os.path.exists(self.download_dir):
            os.makedirs(self.download_dir)
            
    def init_driver(self):
        """Initialize Chrome driver with custom options"""
        options = webdriver.ChromeOptions()
        options.add_experimental_option("prefs", {
            "download.default_directory": self.download_dir,
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing.enabled": True
        })
        
        # Add additional options for stability
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-gpu")
        
        service = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=service, options=options)
        self.driver.maximize_window()
        
    def wait_for_download(self, timeout=60):
        """Wait for download to complete"""
        seconds = 0
        dl_wait = True
        while dl_wait and seconds < timeout:
            time.sleep(1)
            dl_wait = False
            for fname in os.listdir(self.download_dir):
                if fname.endswith('.crdownload'):
                    dl_wait = True
            seconds += 1
        return seconds < timeout
        
    def export_balance_changes(self, account_address):
        """Export balance changes for specified account"""
        self.init_driver()
        
        # Navigate to account page
        url = f"https://solscan.io/account/{account_address}#balanceChanges"
        self.driver.get(url)
        
        wait = WebDriverWait(self.driver, 20)
        
        # Wait for page to fully load
        time.sleep(5)
        
        
        balance_tab = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(text(), 'Balance Changes')]")
        ))
        balance_tab.click()
        print("Clicked Balance Changes tab")
        
        # Wait for and click Export CSV button
        export_button = wait.until(EC.element_to_be_clickable((
            By.XPATH, 
            "//button[.//svg[contains(@class, 'lucide-hard-drive-download')]]"
        )))
        export_button.click()
        print("Clicked Export CSV button")
        
        # Wait for download to complete
        if self.wait_for_download():
            print("Download completed successfully")
            
            # Rename the file with timestamp
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            for fname in os.listdir(self.download_dir):
                if fname.endswith('.csv'):
                    old_path = os.path.join(self.download_dir, fname)
                    new_name = f"solscan_balance_{account_address[:8]}_{timestamp}.csv"
                    new_path = os.path.join(self.download_dir, new_name)
                    os.rename(old_path, new_path)
                    print(f"File saved as: {new_name}")
                    return new_path
        else:
            print("Download timed out")
         
                
    def monitor_account(self, account_address, interval_minutes=5):
        """Monitor account for balance changes at regular intervals"""
        while True:
                print(f"\nChecking balance changes for {account_address}")
                self.export_balance_changes(account_address)
                print(f"Waiting {interval_minutes} minutes before next check...")
                time.sleep(interval_minutes * 60)
           
def download_balance_changes(account_address):
    # Setup Chrome options
    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
        "download.default_directory": os.getcwd(),
        "download.prompt_for_download": False,
    })
    
    # Initialize driver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    
    try:
        # Navigate to the account page
        url = f"https://solscan.io/account/{account_address}#balanceChanges"
        print(f"Navigating to {url}")
        driver.get(url)
        
        # Wait for page to load
        time.sleep(5)
        
        # Wait for the export button with specific content and icon
        export_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH, 
                "//button[.//svg[contains(@class, 'lucide-hard-drive-download')] and contains(.//div, 'Export CSV')]"
            ))
        )
        
        print("Found Export CSV button")
        
        # Click the button
        driver.execute_script("arguments[0].click();", export_button)
        print("Clicked Export CSV button")
        
        # Wait for download
        time.sleep(5)
        
        # Check if file downloaded
        downloads = [f for f in os.listdir() if f.endswith('.csv')]
        if downloads:
            print(f"Successfully downloaded: {downloads[0]}")
        else:
            print("No CSV file found")
            
    except Exception as e:
        print(f"Error occurred: {e}")
        driver.save_screenshot("error.png")
    finally:
        driver.quit()


# Example usage
if __name__ == "__main__":
    # Set up scraper with custom download directory (optional)
    scraper = SolscanScraper(download_dir="./solscan_exports")
    
    # Account to monitor
    account = "HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K"
    
    # Single export
    # scraper.export_balance_changes(account)
    # Example usage
    account = "HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K"
    download_balance_changes(account)

Navigating to https://solscan.io/account/HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K#balanceChanges
Error occurred: Message: 
Stacktrace:
0   chromedriver                        0x000000011054de82 chromedriver + 6696578
1   chromedriver                        0x0000000110545c9a chromedriver + 6663322
2   chromedriver                        0x000000010ff4de3e chromedriver + 405054
3   chromedriver                        0x000000010ff9e815 chromedriver + 735253
4   chromedriver                        0x000000010ff9ed11 chromedriver + 736529
5   chromedriver                        0x000000010ffe9314 chromedriver + 1041172
6   chromedriver                        0x000000010ffc389d chromedriver + 886941
7   chromedriver                        0x000000010ffe63f4 chromedriver + 1029108
8   chromedriver                        0x000000010ffc3613 chromedriver + 886291
9   chromedriver                        0x000000010ff90950 chromedriver + 678224
10  chromedriver                        0x00000

In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
import os

def download_balance_changes(account_address):
    # Setup Chrome options
    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
        "download.default_directory": os.getcwd(),
        "download.prompt_for_download": False,
    })
    
    # Initialize driver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    
    try:
        # Navigate to the account page
        url = f"https://solscan.io/account/{account_address}#balanceChanges"
        print(f"Navigating to {url}")
        driver.get(url)
        
        # Wait for initial page load
        time.sleep(5)
        print("Page loaded")
        
        # Click Balance Changes tab if needed
        try:
            balance_tab = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Balance Changes')]"))
            )
            balance_tab.click()
            print("Clicked Balance Changes tab")
        except:
            print("Balance Changes tab already active")
        
        # Scroll down to make sure we can see the table
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(2)
        
        # Try to find the export button in the table header area
        try:
            export_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH, 
                    "//div[contains(@class, 'data-table')]//button[.//svg[contains(@class, 'lucide-hard-drive-download')]]"
                ))
            )
            print("Found Export CSV button")
            driver.execute_script("arguments[0].scrollIntoView(true);", export_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", export_button)
            print("Clicked Export CSV button")
            
        except TimeoutException:
            print("Export button not found in initial location, trying to scroll more")
            
            # Try scrolling more and looking in different areas
            scroll_heights = [300, 500, 700, 1000]
            for height in scroll_heights:
                driver.execute_script(f"window.scrollBy(0, {height});")
                time.sleep(1)
                try:
                    export_button = driver.find_element(By.XPATH, 
                        "//button[.//svg[contains(@class, 'lucide-hard-drive-download')]]"
                    )
                    print(f"Found button after scrolling {height}px")
                    driver.execute_script("arguments[0].click();", export_button)
                    print("Clicked Export CSV button")
                    break
                except:
                    continue
        
        # Wait for download
        time.sleep(5)
        
        # Check if file downloaded
        downloads = [f for f in os.listdir() if f.endswith('.csv')]
        if downloads:
            print(f"Successfully downloaded: {downloads[0]}")
        else:
            print("No CSV file found")
            driver.save_screenshot("no_download.png")
            
    except Exception as e:
        print(f"Error occurred: {e}")
        driver.save_screenshot("error.png")
    finally:
        driver.quit()

# Example usage
account = "HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K"
download_balance_changes(account)

Navigating to https://solscan.io/account/HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K#balanceChanges
Page loaded
Balance Changes tab already active
Export button not found in initial location, trying to scroll more
No CSV file found


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
import os

def download_balance_changes(account_address):
    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
        "download.default_directory": os.getcwd(),
        "download.prompt_for_download": False,
    })
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    
    try:
        # Navigate to the account page
        url = f"https://solscan.io/account/{account_address}#balanceChanges"
        print(f"Navigating to {url}")
        driver.get(url)
        time.sleep(5)
        
        # Click Balance Changes tab
        try:
            balance_tab = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Balance Changes')]"))
            )
            balance_tab.click()
            print("Clicked Balance Changes tab")
            time.sleep(2)
        except:
            print("Balance Changes tab already active")
        
        # Scroll down
        driver.execute_script("window.scrollBy(0, 300);")
        time.sleep(2)
        
        # Click Export CSV button
        export_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[.//div[contains(text(), 'Export CSV')]]"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", export_button)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", export_button)
        print("Clicked Export CSV button")
        
        # Wait 30 seconds for manual inspection of popup
        print("Waiting 30 seconds for popup inspection...")
        time.sleep(30)
        
        # Keep browser open
        input("Press Enter to close the browser...")
            
    except Exception as e:
        print(f"Error occurred: {e}")
        driver.save_screenshot("error.png")
        
    finally:
        driver.quit()

# Run it
if __name__ == "__main__":
    account = "HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K"
    download_balance_changes(account)

Navigating to https://solscan.io/account/HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K#balanceChanges
Balance Changes tab already active
Clicked Export CSV button
Waiting 30 seconds for popup inspection...


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
import os

def download_balance_changes(account_address):
    # Get current directory
    current_dir = os.getcwd()
    print(f"Downloads will be saved to: {current_dir}")

    options = webdriver.ChromeOptions()
    # Set download directory to current directory
    prefs = {
        "download.default_directory": current_dir,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }
    options.add_experimental_option("prefs", prefs)
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    
    try:
        # Navigate to the account page
        url = f"https://solscan.io/account/{account_address}#balanceChanges"
        print(f"Navigating to {url}")
        driver.get(url)
        time.sleep(5)
        
        # Click Balance Changes tab
        try:
            balance_tab = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Balance Changes')]"))
            )
            balance_tab.click()
            print("Clicked Balance Changes tab")
            time.sleep(2)
        except:
            print("Balance Changes tab already active")
        
        # Scroll down
        driver.execute_script("window.scrollBy(0, 300);")
        time.sleep(2)
        
        # Click Export CSV button
        export_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[.//div[contains(text(), 'Export CSV')]]"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", export_button)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", export_button)
        print("Clicked Export CSV button")
        
        # Wait for popup and click Download button
        download_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((
                By.XPATH, 
                "//div[@role='dialog']//button[contains(text(), 'Download')]"
            ))
        )
        time.sleep(1)
        driver.execute_script("arguments[0].click();", download_button)
        print("Clicked Download button in popup")
        
        # Wait for download
        time.sleep(5)
        
        # Check downloads in current directory
        downloads = [f for f in os.listdir(current_dir) if f.endswith('.csv')]
        if downloads:
            print(f"Successfully downloaded: {downloads[0]}")
            print(f"File location: {os.path.join(current_dir, downloads[0])}")
        else:
            print("No CSV file found")
            driver.save_screenshot("no_download.png")
            
    except Exception as e:
        print(f"Error occurred: {e}")
        driver.save_screenshot("error.png")
        
    finally:
        driver.quit()

# Run it
if __name__ == "__main__":
    account = "HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K"
    download_balance_changes(account)

Downloads will be saved to: /Users/poj/sol-transaction
Navigating to https://solscan.io/account/HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K#balanceChanges
Balance Changes tab already active
Clicked Export CSV button
Clicked Download button in popup
Successfully downloaded: export_balance_change_HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K_1733392020668.csv
File location: /Users/poj/sol-transaction/export_balance_change_HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K_1733392020668.csv


In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import time
import os

def get_latest_csv():
    # Get all CSV files in current directory
    csv_files = [f for f in os.listdir() if f.endswith('.csv')]
    if not csv_files:
        return None
        
    # Get the most recently modified CSV file
    latest_csv = max(csv_files, key=lambda x: os.path.getmtime(x))
    return latest_csv

def analyze_token_holdings(csv_file, token_address, min_amount=100000):
    print(f"Analyzing file: {csv_file}")
    
    # Read CSV file
    df = pd.read_csv(csv_file)
    
    # Convert BlockTime to datetime
    df['BlockTime'] = pd.to_datetime(df['BlockTime'])
    
    # Filter for specific token
    token_df = df[df['TokenAddress'] == token_address].copy()
    if token_df.empty:
        print(f"No transactions found for token {token_address}")
        return
    
    # Sort by BlockTime
    token_df = token_df.sort_values('BlockTime')
    
    # Calculate running balance
    current_balance = 0
    holding_periods = []
    holding_start = None
    
    for index, row in token_df.iterrows():
        if row['ChangeType'] == 'inc':
            current_balance += float(row['ChangeAmount'])
        else:
            current_balance -= float(row['ChangeAmount'])
        
        # Check if balance crosses threshold
        if current_balance >= min_amount:
            if holding_start is None:
                holding_start = row['BlockTime']
        else:
            if holding_start is not None:
                holding_end = row['BlockTime']
                days = (holding_end - holding_start).days
                holding_periods.append({
                    'start': holding_start,
                    'end': holding_end,
                    'days': days,
                    'amount': current_balance
                })
                holding_start = None
    
    # Check if still holding
    if holding_start is not None:
        holding_end = datetime.now()
        days = (holding_end - holding_start).days
        holding_periods.append({
            'start': holding_start,
            'end': holding_end,
            'days': days,
            'amount': current_balance
        })
    
    # Print results
    if holding_periods:
        for period in holding_periods:
            if period['days'] >= 20:
                print(f"\nFound holding period over 20 days:")
                print(f"Start date: {period['start'].strftime('%Y-%m-%d')}")
                print(f"End date: {period['end'].strftime('%Y-%m-%d')}")
                print(f"Days held: {period['days']}")
                print(f"Amount held: {period['amount']}")
                print("---")
    else:
        print(f"\nNo periods found where token balance exceeded {min_amount} for 20 days or more")

# Wait for latest downloaded CSV and analyze it
def main():
    # Wait for the downloaded file
    print("Waiting for CSV file download...")
    time.sleep(5)  # Wait for download to complete
    
    # Get latest CSV file
    csv_file = get_latest_csv()
    if csv_file is None:
        print("No CSV file found!")
        return
        
    # Analyze the token
    token_address = "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump"
    analyze_token_holdings(csv_file, token_address, 100000)

if __name__ == "__main__":
    main()

Waiting for CSV file download...
Analyzing file: export_balance_change_HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K_1733392142942.csv


KeyError: 'BlockTime'

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
import os

def download_balance_changes(account_address):
    # Get current directory
    current_dir = os.getcwd()
    print(f"Downloads will be saved to: {current_dir}")

    options = webdriver.ChromeOptions()
    # Set download directory to current directory
    prefs = {
        "download.default_directory": current_dir,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }
    options.add_experimental_option("prefs", prefs)
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    
    try:
        # Navigate to the account page
        url = f"https://solscan.io/account/{account_address}#balanceChanges"
        print(f"Navigating to {url}")
        driver.get(url)
        time.sleep(5)
        
        # Click Balance Changes tab
        try:
            balance_tab = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Balance Changes')]"))
            )
            balance_tab.click()
            print("Clicked Balance Changes tab")
            time.sleep(2)
        except:
            print("Balance Changes tab already active")
        
        # Scroll down
        driver.execute_script("window.scrollBy(0, 300);")
        time.sleep(2)
        
        # Click Export CSV button
        export_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[.//div[contains(text(), 'Export CSV')]]"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", export_button)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", export_button)
        print("Clicked Export CSV button")
        
        # Wait for popup and click Download button
        download_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((
                By.XPATH, 
                "//div[@role='dialog']//button[contains(text(), 'Download')]"
            ))
        )
        time.sleep(1)
        driver.execute_script("arguments[0].click();", download_button)
        print("Clicked Download button in popup")
        
        # Wait for download
        time.sleep(5)
        
        # Check downloads in current directory
        downloads = [f for f in os.listdir(current_dir) if f.endswith('.csv')]
        if downloads:
            print(f"Successfully downloaded: {downloads[0]}")
            print(f"File location: {os.path.join(current_dir, downloads[0])}")
        else:
            print("No CSV file found")
            driver.save_screenshot("no_download.png")
            
    except Exception as e:
        print(f"Error occurred: {e}")
        driver.save_screenshot("error.png")
        
    finally:
        driver.quit()

# Run it
if __name__ == "__main__":
    account = "HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K"
    download_balance_changes(account)

Downloads will be saved to: /Users/poj/sol-transaction
Navigating to https://solscan.io/account/HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K#balanceChanges
Balance Changes tab already active
Clicked Export CSV button
Clicked Download button in popup
Successfully downloaded: export_balance_change_HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K_1733392020668.csv
File location: /Users/poj/sol-transaction/export_balance_change_HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K_1733392020668.csv


In [9]:
import pandas as pd
from datetime import datetime, timedelta
import os

def get_latest_csv():
    # Get all CSV files in current directory
    csv_files = [f for f in os.listdir() if f.endswith('.csv')]
    if not csv_files:
        return None
    # Get the most recently modified CSV file
    latest_csv = max(csv_files, key=lambda x: os.path.getmtime(x))
    return latest_csv

def analyze_token_holdings(csv_file, token_address, min_amount=100000):
    print(f"Analyzing file: {csv_file}")

    # Read CSV file
    df = pd.read_csv(csv_file)

    # Clean up column names by stripping whitespace
    df.columns = df.columns.str.strip()

    # Convert BlockTime to datetime
    df['BlockTime'] = pd.to_datetime(df['BlockTime'])

    # Filter for specific token
    token_df = df[df['TokenAddress'].str.strip() == token_address].copy()
    if token_df.empty:
        print(f"No transactions found for token {token_address}")
        return

    # Sort by BlockTime
    token_df = token_df.sort_values('BlockTime')

    # Calculate holding periods
    holding_periods = {}
    current_balance = 0
    holding_start = None

    for index, row in token_df.iterrows():
        if row['ChangeType'].strip() == 'inc':
            current_balance += float(row['ChangeAmount'])
        else:
            current_balance -= float(row['ChangeAmount'])

        # Check if balance crosses threshold
        if current_balance >= min_amount:
            if holding_start is None:
                holding_start = row['BlockTime']
        else:
            if holding_start is not None:
                holding_end = row['BlockTime']
                days = (holding_end - holding_start).days
                user = row['TokenAccount']
                if user not in holding_periods:
                    holding_periods[user] = []
                holding_periods[user].append({
                    'start': holding_start,
                    'end': holding_end,
                    'days': days,
                    'amount': current_balance
                })
                holding_start = None

    # Check if still holding
    if holding_start is not None:
        holding_end = datetime.now()
        days = (holding_end - holding_start).days
        user = token_df.iloc[-1]['TokenAccount']
        if user not in holding_periods:
            holding_periods[user] = []
        holding_periods[user].append({
            'start': holding_start,
            'end': holding_end,
            'days': days,
            'amount': current_balance
        })

    # Print results
    for user, periods in holding_periods.items():
        print(f"\nUser: {user}")
        for period in periods:
            if period['days'] >= 20:
                print(f"Start date: {period['start'].strftime('%Y-%m-%d')}")
                print(f"End date: {period['end'].strftime('%Y-%m-%d')}")
                print(f"Days held: {period['days']}")
                print(f"Amount held: {period['amount']}")
                print("---")

def main():
    # Wait for the downloaded file
    print("Waiting for CSV file download...")
    # Get latest CSV file
    csv_file = get_latest_csv()
    if csv_file is None:
        print("No CSV file found!")
        return
    # Analyze the token
    token_address = "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump"
    analyze_token_holdings(csv_file, token_address, 100000)

if __name__ == "__main__":
    main()

Waiting for CSV file download...
Analyzing file: export_balance_change_HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K_1733392983543.csv

User: 6WmNyaTZXAr7HuvEGyFUVPxiJJ3sKhGjoUdThuZhWf4k


In [23]:
import pandas as pd
from datetime import datetime, timedelta
import os

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
import os

def download_balance_changes(account_address):
    # Get current directory
    current_dir = os.getcwd()
    print(f"Downloads will be saved to: {current_dir}")

    options = webdriver.ChromeOptions()
    # Set download directory to current directory
    prefs = {
        "download.default_directory": current_dir,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }
    options.add_experimental_option("prefs", prefs)
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    
    try:
        # Navigate to the account page
        url = f"https://solscan.io/account/{account_address}#balanceChanges"
        print(f"Navigating to {url}")
        driver.get(url)
        time.sleep(5)
        
        # Click Balance Changes tab
        try:
            balance_tab = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Balance Changes')]"))
            )
            balance_tab.click()
            print("Clicked Balance Changes tab")
            time.sleep(2)
        except:
            print("Balance Changes tab already active")
        
        # Scroll down
        driver.execute_script("window.scrollBy(0, 300);")
        time.sleep(2)
        
        # Click Export CSV button
        export_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[.//div[contains(text(), 'Export CSV')]]"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", export_button)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", export_button)
        print("Clicked Export CSV button")
        
        # Wait for popup and click Download button
        download_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((
                By.XPATH, 
                "//div[@role='dialog']//button[contains(text(), 'Download')]"
            ))
        )
        time.sleep(1)
        driver.execute_script("arguments[0].click();", download_button)
        print("Clicked Download button in popup")
        
        # Wait for download
        time.sleep(5)
        
        # Check downloads in current directory
        downloads = [f for f in os.listdir(current_dir) if f.endswith('.csv')]
        if downloads:
            print(f"Successfully downloaded: {downloads[0]}")
            print(f"File location: {os.path.join(current_dir, downloads[0])}")
        else:
            print("No CSV file found")
            driver.save_screenshot("no_download.png")
            
    except Exception as e:
        print(f"Error occurred: {e}")
        driver.save_screenshot("error.png")
        
    finally:
        driver.quit()

def get_latest_csv():
    # Get all CSV files in the current directory
    csv_files = [f for f in os.listdir() if f.endswith('.csv')]
    if not csv_files:
        return None
    # Get the most recently modified CSV file
    latest_csv = max(csv_files, key=lambda x: os.path.getmtime(x))
    return latest_csv

def calculate_daily_balances(csv_file, token_address, threshold=100000):
    print(f"Analyzing file: {csv_file}")

    # Read CSV file
    df = pd.read_csv(csv_file)

    # Clean up column names by stripping whitespace
    df.columns = df.columns.str.strip()

    # Convert BlockTime to datetime
    df['BlockTime'] = pd.to_datetime(df['BlockTime'])

    # Filter for specific token
    token_df = df[df['TokenAddress'].str.strip() == token_address].copy()
    if token_df.empty:
        print(f"No transactions found for token {token_address}")
        return

    # Sort by BlockTime
    token_df = token_df.sort_values('BlockTime')

    # Track daily balances
    daily_balances = {}

    # Track current balance
    current_balance = 0

    # Track how many days the balance is greater than the threshold
    days_above_threshold = 0

    for index, row in token_df.iterrows():
        # Update balance based on ChangeType
        if row['ChangeType'].strip() == 'inc':
            current_balance += float(row['ChangeAmount'])
        else:
            current_balance -= float(row['ChangeAmount'])

        # Get the date of the transaction
        transaction_date = row['BlockTime'].date()

        # Store the balance for the specific date
        if transaction_date not in daily_balances:
            daily_balances[transaction_date] = 0

        # Update the balance for that day
        daily_balances[transaction_date] += current_balance

    # Count how many days the balance is greater than the threshold
    for date, balance in daily_balances.items():
        if balance > threshold:
            days_above_threshold += 1

    # Display results in a calendar-like format
    print("\nDaily Token Balances:")
    start_date = min(daily_balances.keys())
    end_date = max(daily_balances.keys())
    
    current_date = start_date
    while current_date <= end_date:
        balance = daily_balances.get(current_date, 0)
        print(f"Date: {current_date} - Balance: {balance}")
        current_date += timedelta(days=1)

    # Print the count of days with balance above threshold
    print(f"\nTotal number of days with balance greater than {threshold}: {days_above_threshold}")
    
def calculate_consecutive_days(csv_file, token_address, threshold=100000):
    print(f"Analyzing file: {csv_file}")

    # Read CSV file
    df = pd.read_csv(csv_file)

    # Clean up column names by stripping whitespace
    df.columns = df.columns.str.strip()

    # Convert BlockTime to datetime
    df['BlockTime'] = pd.to_datetime(df['BlockTime'])

    # Filter for specific token
    token_df = df[df['TokenAddress'].str.strip() == token_address].copy()
    if token_df.empty:
        print(f"No transactions found for token {token_address}")
        return

    # Sort by BlockTime
    token_df = token_df.sort_values('BlockTime')

    # Initialize variables for tracking consecutive days
    consecutive_days = 0
    total_consecutive_days = 0
    is_above_threshold = False
    last_valid_date = None
    current_balance = 0

    print("\nAnalyzing transactions for token:", token_address)

    # Loop through the token transactions to calculate consecutive days above threshold
    for index, row in token_df.iterrows():
        current_post_balance = row['PostBalancer']/10**6
        transaction_date = row['BlockTime'].date()

        # Print debugging information for each transaction
        print(f"Transaction {index + 1}: Date = {transaction_date}, PostBalancer = {current_post_balance}")

        # If balance is above threshold, count the days
        if current_post_balance >= threshold:
            if not is_above_threshold:
                # Start counting days when crossing the threshold
                print(f"Crossed threshold of {threshold} on {transaction_date}. Starting count.")
                is_above_threshold = True
                last_valid_date = transaction_date
                consecutive_days = 1  # Count this day as the first day above threshold
            else:
                # Continue counting if still above threshold
                days_diff = (transaction_date - last_valid_date).days
                consecutive_days += days_diff
                print(f"Continuing count. Days diff: {days_diff}. Consecutive days: {consecutive_days}")
            last_valid_date = transaction_date

        elif is_above_threshold:
            # If balance goes below threshold, finalize counting, add days between
            if last_valid_date:
                days_diff = (transaction_date - last_valid_date).days
                consecutive_days += days_diff
                print(f"PostBalancer dropped below threshold on {transaction_date}. Adding days: {days_diff}. Total consecutive days: {consecutive_days}")

            total_consecutive_days += consecutive_days
            is_above_threshold = False
            consecutive_days = 0
            last_valid_date = None

    # If still above threshold at the end, add the remaining days
    if is_above_threshold:
        print(f"Still above threshold. Adding remaining consecutive days.")
        total_consecutive_days += consecutive_days

    # Display results
    print("\nTotal consecutive days with PostBalancer greater than threshold:", total_consecutive_days)



def main():
    
    account = "HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K"
    download_balance_changes(account)
    # account = "B9BAojYpky9zSMf8qmnfq7ThD9jZw41gQnZ1xf8qYKx8"
    # download_balance_changes(account)
    # Wait for the downloaded file
    print("Waiting for CSV file download...")
    # Get latest CSV file
    csv_file = get_latest_csv()
    if csv_file is None:
        print("No CSV file found!")
        return

    # Analyze the token
    token_address = "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump"
    calculate_consecutive_days(csv_file, token_address)

if __name__ == "__main__":
    main()


Downloads will be saved to: /Users/poj/sol-transaction
Navigating to https://solscan.io/account/HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K#balanceChanges
Balance Changes tab already active
Clicked Export CSV button
Clicked Download button in popup
Successfully downloaded: export_balance_change_HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K_1733395178997.csv
File location: /Users/poj/sol-transaction/export_balance_change_HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K_1733395178997.csv
Waiting for CSV file download...
Analyzing file: export_balance_change_HSH56eLeftiyXk5ubueH5oJ7iqzNwXgVgsDjouPBu85K_1733396787806.csv

Analyzing transactions for token: EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump
Transaction 47: Date = 2024-11-27, PostBalancer = 583088.230565
Crossed threshold of 100000 on 2024-11-27. Starting count.
Transaction 45: Date = 2024-11-27, PostBalancer = 1703477.339546
Continuing count. Days diff: 0. Consecutive days: 1
Transaction 36: Date = 2024-11-28, PostBalancer = 9037095.5

In [ ]:
import unittest
import pandas as pd
from datetime import datetime

# ฟังก์ชันที่ต้องการทดสอบ (ฟังก์ชันที่เราเขียนก่อนหน้านี้)
def calculate_consecutive_days(df, token_address, threshold=100000):
    token_df = df[df['TokenAddress'].str.strip() == token_address].copy()
    if token_df.empty:
        return 0

    # Sort by BlockTime
    token_df = token_df.sort_values('BlockTime')

    # Initialize variables for tracking consecutive days
    consecutive_days = 0
    total_consecutive_days = 0
    is_above_threshold = False
    last_valid_date = None
    current_balance = 0

    # Loop through the token transactions to calculate consecutive days above threshold
    for index, row in token_df.iterrows():
        current_post_balance = row['PostBalancer']
        transaction_date = row['BlockTime'].date()

        # If balance is above threshold, count the days
        if current_post_balance >= threshold:
            if not is_above_threshold:
                # Start counting days when crossing the threshold
                is_above_threshold = True
                last_valid_date = transaction_date
                consecutive_days = 1  # Count this day as the first day above threshold
            else:
                # Continue counting if still above threshold
                days_diff = (transaction_date - last_valid_date).days
                consecutive_days += days_diff
            last_valid_date = transaction_date

        elif is_above_threshold:
            # If balance goes below threshold, finalize counting
            if last_valid_date:
                days_diff = (transaction_date - last_valid_date).days
                consecutive_days += days_diff

            total_consecutive_days += consecutive_days
            is_above_threshold = False
            consecutive_days = 0
            last_valid_date = None

    # If still above threshold at the end, add the remaining days
    if is_above_threshold:
        total_consecutive_days += consecutive_days

    return total_consecutive_days


class TestCalculateConsecutiveDays(unittest.TestCase):

    def setUp(self):
        # Prepare test data (mock data)
        self.test_data = [
            {"BlockTime": "2024-11-28 00:00:00", "PostBalancer": 8342367935286, "TokenAddress": "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump"},
            {"BlockTime": "2024-12-01 00:00:00", "PostBalancer": 8342367935286, "TokenAddress": "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump"},
            {"BlockTime": "2024-12-04 00:00:00", "PostBalancer": 0, "TokenAddress": "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump"},
            {"BlockTime": "2024-12-05 00:00:00", "PostBalancer": 200000, "TokenAddress": "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump"},
            {"BlockTime": "2024-12-10 00:00:00", "PostBalancer": 0, "TokenAddress": "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump"}
        ]
        # Convert to pandas DataFrame
        self.df = pd.DataFrame(self.test_data)
        self.df['BlockTime'] = pd.to_datetime(self.df['BlockTime'])

    def test_consecutive_days_with_threshold(self):
        # Expected 6 days from 2024-11-28 to 2024-12-03
        result = calculate_consecutive_days(self.df, "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump", threshold=100000)
        self.assertEqual(result, 6)

    def test_multiple_entries_below_threshold(self):
        # Test when balance goes below the threshold after crossing it
        result = calculate_consecutive_days(self.df, "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump", threshold=100000)
        self.assertEqual(result, 6)

    def test_cross_threshold_multiple_times(self):
        # Test case where balance crosses threshold multiple times
        test_data_cross = [
            {"BlockTime": "2024-11-28 00:00:00", "PostBalancer": 8342367935286, "TokenAddress": "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump"},
            {"BlockTime": "2024-12-01 00:00:00", "PostBalancer": 8342367935286, "TokenAddress": "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump"},
            {"BlockTime": "2024-12-04 00:00:00", "PostBalancer": 0, "TokenAddress": "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump"},
            {"BlockTime": "2024-12-05 00:00:00", "PostBalancer": 200000, "TokenAddress": "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump"},
            {"BlockTime": "2024-12-10 00:00:00", "PostBalancer": 0, "TokenAddress": "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump"}
        ]
        df_cross = pd.DataFrame(test_data_cross)
        df_cross['BlockTime'] = pd.to_datetime(df_cross['BlockTime'])
        result = calculate_consecutive_days(df_cross, "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump", threshold=100000)
        self.assertEqual(result, 13)

    def test_no_transactions_above_threshold(self):
        # Test when no transaction is above the threshold
        test_data_no_above = [
            {"BlockTime": "2024-11-28 00:00:00", "PostBalancer": 50000, "TokenAddress": "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump"},
            {"BlockTime": "2024-12-01 00:00:00", "PostBalancer": 75000, "TokenAddress": "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump"},
            {"BlockTime": "2024-12-04 00:00:00", "PostBalancer": 90000, "TokenAddress": "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump"},
        ]
        df_no_above = pd.DataFrame(test_data_no_above)
        df_no_above['BlockTime'] = pd.to_datetime(df_no_above['BlockTime'])
        result = calculate_consecutive_days(df_no_above, "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump", threshold=100000)
        self.assertEqual(result, 0)

    def test_no_transactions(self):
        # Test when there are no transactions at all
        df_empty = pd.DataFrame(columns=["BlockTime", "PostBalancer", "TokenAddress"])
        result = calculate_consecutive_days(df_empty, "EStPXF2Mh3NVEezeysYfhrWXnuqwmbmjqLSP9vR5pump", threshold=100000)
        self.assertEqual(result, 0)


if __name__ == "__main__":
    # Run the unit tests
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


FFF..
FAIL: test_consecutive_days_with_threshold (__main__.TestCalculateConsecutiveDays)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/8s/d58d7k4x6r36pnw83v9pnvg00000gn/T/ipykernel_4405/565463013.py", line 75, in test_consecutive_days_with_threshold
    self.assertEqual(result, 6)
AssertionError: 13 != 6

FAIL: test_cross_threshold_multiple_times (__main__.TestCalculateConsecutiveDays)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/8s/d58d7k4x6r36pnw83v9pnvg00000gn/T/ipykernel_4405/565463013.py", line 94, in test_cross_threshold_multiple_times
    self.assertEqual(result, 6)
AssertionError: 13 != 6

FAIL: test_multiple_entries_below_threshold (__main__.TestCalculateConsecutiveDays)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/8s/d58d7k4x6r36pnw83v